In [2]:
# Dependencies
import gmaps

# Google developer API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [3]:
from ipynb.fs.full.WeatherPy import data_df

ImportError: Could not import C:\Users\izzyb\OneDrive\Documents\GitHub\python-api-challenge\VacationPy\WeatherPy.ipynb for ipynb.fs.full.WeatherPy: not a valid ipynb file